In [2]:
from datetime import datetime, timedelta
import pandas as pd



click_stream = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/click_stream_final.csv')
transaction = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/transaction_final.csv')
product = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/product_final.csv')
customer = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/customer_final.csv')

tem = pd.merge(click_stream[['session_id', 'event_time', 'event_name']], transaction[['session_id', 'customer_id']].drop_duplicates(), on = 'session_id', how = "left")
tem2 = pd.merge(tem[tem.customer_id.notnull()], customer[['customer_id', 'first_join_date']], on = 'customer_id', how= 'left')




In [41]:
#함수 사용에 필요한 특성
#click_stream
#   event_name: 'HOMEPAGE' 인 이벤트만 조회하기 때문에
#   event_time: 재방문에 걸린 시간 조회하기 위해
#   session_id: transaction 테이블과 병합하여 customer_id를 얻기 위해
#transaction
#   session_id: click_stream 테이블과 병합하여 customer_id를 얻기 위해
#   customer_id: customer 테이블과 병합하여 first_join_date 를 얻기 위해
#customer
#   customer_id: click_stream 테이블과 병합하여 first_join_date 를 얻기 위해
#   first_join_date: 가입 시기를 알기 위해


def bounce_judge(df):
    #측정 기간 설정
    start_date = pd.to_datetime('2020-07-01')
    end_date = pd.to_datetime('2022-06-30')
    
    #측정 기간에 해당하는 월 수: 24개
    num_of_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
    
    #이탈 판단 기준(10일 = 60 * 60 * 24 * 10 = 864000 초)
    criterion = 864000

    # 월별 컬럼 이름 생성
    column_names = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m') for i in range(num_of_months)]

    #데이터의 시간 데이터 타입 초기화
    df.event_time = pd.to_datetime(df.event_time)

    # 빈 데이터프레임 생성
    res = pd.DataFrame(columns=column_names)

    #customer_id를 기준으로 반복
    for customer in df.customer_id.unique():
        
        #베이스 딕셔너리 생성
        tem = {month : 0 for month in sorted(column_names)}

        #조건에 맞는 데이터로그 불러오기
        log = df[(df.customer_id == customer) & (df.event_name == 'HOMEPAGE') & (pd.to_datetime(df.event_time.dt.date) <= end_date) & (pd.to_datetime(df.event_time.dt.date) >= start_date)].sort_values('event_time').reset_index(drop = True)

        # 정해진 기간에 데이터가 없을 경우 log.loc[0, 'first_join_date'] 를 호출 하는 과정에서 오류 발생 함. 이를 방지하고자 데이터의 개수가 0개이면 넘어감
        if len(log) == 0:
            pass

        else:    # 2020-07월 이전에 계정을 생성한 경우
            if pd.to_datetime(log.loc[0, 'first_join_date']) < start_date:

                #방문 기록이 없는 달 추출하여 집합으로 저장
                date_set = set([date.strftime('%Y-%m') for date in log.event_time])
                bounce_month = set(column_names) - date_set
        
                #homepage 재방문 시간 계산
                log['diff'] = log.event_time.diff().dt.total_seconds()

                #첫번째 인덱스에는 start_date 부터의 시간 간격 측정
                log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp('2020-07-01 0:0:0', tz= 'UTC')).total_seconds()

                #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
                bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=10)]))

            # 2020-07월 이후에 계정을 생성한 경우
            else:
                
                #start_date 이후 first_join_date 이전 월 추출(이탈 아닌것으로 확인해야하기 때문에)
                date_range = pd.date_range(start=start_date, end=log.first_join_date.iloc[0], freq='M')
                extracted_months = set(date.strftime('%Y-%m') for date in date_range)

                #방문 기록이 없는 달 추출하여 집합으로 저장
                date_set = set([date.strftime('%Y-%m') for date in log.event_time])
                bounce_month = set(column_names) - date_set - extracted_months
                    
                #homepage 재방문 시간 계산
                log['diff'] = log.event_time.diff().dt.total_seconds()

                #첫번째 인덱스에는 first_join_date 부터의 시간 간격 측정
                log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp(log.loc[0, 'first_join_date'], tz= 'UTC')).total_seconds()

                #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
                bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=10)]))

        #이탈(1) 입력
        for month in bounce_month:
            tem[month] = 1
    
        #customer_id 와 key에 대해 이탈여부 매칭하는 데이터프레임 만들기
        for key in tem:
            res.loc[customer, key] = tem[key]
    
    return res

In [21]:
start_date = pd.to_datetime('2020-07-01')
end_date = pd.to_datetime('2022-06-30')
    
tem2[(tem2.customer_id == 1) & (tem2.event_name == 'HOMEPAGE') & (pd.to_datetime(tem2.event_time.dt.date) <= end_date) & (pd.to_datetime(tem2.event_time.dt.date) >= start_date)].sort_values('event_time').reset_index(drop = True)

,session_id,event_time,event_name,customer_id,first_join_date


In [42]:
bounce_judge(tem2)

18770.0 

82237.0 91760.0 43202.0 10167.0 29496.0 38588.0 39915.0 29240.0 34965.0 72065.0 9507.0 18123.0 6033.0 52644.0 45253.0 17434.0 36097.0 88736.0 65020.0 69072.0 83272.0 95515.0 8036.0 31461.0 54846.0 81933.0 35627.0 69740.0 41055.0 75728.0 9316.0 22070.0 51793.0 51154.0 61513.0 64659.0 92328.0 95492.0 89660.0 55389.0 49577.0 28940.0 6462.0 63241.0 20991.0 57813.0 60652.0 98034.0 415.0 70130.0 59256.0 21796.0 7848.0 60289.0 27762.0 45584.0 78376.0 70049.0 24270.0 20143.0 87113.0 35811.0 88500.0 21093.0 40786.0 61846.0 21542.0 86416.0 75921.0 5816.0 58549.0 13789.0 

KeyboardInterrupt: 

In [38]:
tem2[tem2.customer_id == 36097.0 ]

,session_id,event_time,event_name,customer_id,first_join_date
90890,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 16:51:07.980481+00:00,HOMEPAGE,36097.0,2019-11-24
90891,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 16:56:41.980481+00:00,ADD_TO_CART,36097.0,2019-11-24
90892,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 17:01:56.980481+00:00,ADD_PROMO,36097.0,2019-11-24
90893,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 17:32:34.354900+00:00,BOOKING,36097.0,2019-11-24
90894,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 16:56:56.980481+00:00,CLICK,36097.0,2019-11-24
90895,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 17:03:00.980481+00:00,PROMO_PAGE,36097.0,2019-11-24
90896,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 17:08:28.980481+00:00,SCROLL,36097.0,2019-11-24
90897,56b052fd-af23-451d-8472-feb17bfd011d,2019-11-24 17:14:20.980481+00:00,PROMO_PAGE,36097.0,2019-11-24
